In [ ]:
#importing the libraries
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib

Using matplotlib backend: <object object at 0x7f80b08e36a0>


#1.Load the given dataset into a DataFrame

In [ ]:
#loading the csv file
df=pd.read_csv("/content/insurance.csv")
df.head(10)

,age,sex,bmi,children,smoker,region,charges
0,19,female,27.900,0,yes,southwest,16884.92400
1,18,male,33.770,1,no,southeast,1725.55230
2,28,male,33.000,3,no,southeast,4449.46200
3,33,male,22.705,0,no,northwest,21984.47061
4,32,male,28.880,0,no,northwest,3866.85520
5,31,female,25.740,0,no,southeast,3756.62160
6,46,female,33.440,1,no,southeast,8240.58960
7,37,female,27.740,3,no,northwest,7281.50560
8,37,male,29.830,2,no,northeast,6406.41070
9,60,female,25.840,0,no,northwest,28923.13692


In [ ]:
#find information
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1338 entries, 0 to 1337
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   age       1338 non-null   int64  
 1   sex       1338 non-null   object 
 2   bmi       1338 non-null   float64
 3   children  1338 non-null   int64  
 4   smoker    1338 non-null   object 
 5   region    1338 non-null   object 
 6   charges   1338 non-null   float64
dtypes: float64(2), int64(2), object(3)
memory usage: 73.3+ KB


In [ ]:
#find shape
df.shape

(1338, 7)

In [ ]:
#find describe
df.describe()

,age,bmi,children,charges
count,1338.000000,1338.000000,1338.000000,1338.000000
mean,39.207025,30.663397,1.094918,13270.422265
std,14.049960,6.098187,1.205493,12110.011237
min,18.000000,15.960000,0.000000,1121.873900
25%,27.000000,26.296250,0.000000,4740.287150
50%,39.000000,30.400000,1.000000,9382.033000
75%,51.000000,34.693750,2.000000,16639.912515
max,64.000000,53.130000,5.000000,63770.428010


#2.Find null values and drop them if it is there

In [ ]:
#find the null values
df.isnull().sum()

age         0
sex         0
bmi         0
children    0
smoker      0
region      0
charges     0
dtype: int64

#3.Find an Outlier and solve it using the interquartile range (IQR) method

In [ ]:
sns.distplot(df["charges"])
sns.boxplot(df["charges"])

<ipython-input-8-799b601923e6>:1: UserWarning: 

`distplot` is a deprecated function and will be removed in seaborn v0.14.0.

Please adapt your code to use either `displot` (a figure-level function with
similar flexibility) or `histplot` (an axes-level function for histograms).

For a guide to updating your code to use the new functions, please see
https://gist.github.com/mwaskom/de44147ed2974457ad6372750bbe5751

  sns.distplot(df["charges"])


<Axes: xlabel='charges', ylabel='Density'>

In [ ]:
hp=sorted(df["charges"])
q1,q3=np.percentile(hp,[25,75])
lower=q1-(1.5*(q3-q1))
upper=q3+(1.5*(q3-q1))
below=df["charges"]>lower
above=df["charges"]<upper
df=df[below & above]
df
df.shape

(1199, 7)

In [ ]:
sns.boxplot(df["charges"])
sns.displot(df["charges"])


In [ ]:
#find data types
df.dtypes

age           int64
sex          object
bmi         float64
children      int64
smoker       object
region       object
charges     float64
dtype: object

In [ ]:
df.sex.value_counts()
df.smoker.value_counts()
df.region.value_counts()

southeast    307
northwest    305
northeast    296
southwest    291
Name: region, dtype: int64

In [ ]:
df_new=df.dropna()
df_new.shape

(1199, 7)

In [ ]:
df_new.columns
df_new.dtypes

age           int64
sex          object
bmi         float64
children      int64
smoker       object
region       object
charges     float64
dtype: object

#4.Do one-hot encoding for categorical features

In [ ]:
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder

In [ ]:
category=df_new.select_dtypes(include=[object])
num=df_new.select_dtypes(include=["float64","int64"])

In [ ]:
#finding the encoding
encoded=category.apply(LabelEncoder().fit_transform)
onehot=pd.DataFrame(OneHotEncoder().fit_transform(encoded).toarray())
onehot.columns=['sex_male','sex_female','smoker_no','smoker_yes','region_northeast','region_northwest','region_southeast','region_southwest']
onehot.head()

,sex_male,sex_female,smoker_no,smoker_yes,region_northeast,region_northwest,region_southeast,region_southwest
0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0
1,0.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0
2,0.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0
3,0.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0
4,0.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0


In [ ]:
#concatenating onehot with other variable
df_onehotencoded=pd.concat([num,onehot.reset_index(drop=True)],axis=1)
df_onehotencoded.head()

,age,bmi,children,charges,sex_male,sex_female,smoker_no,smoker_yes,region_northeast,region_northwest,region_southeast,region_southwest
0,19.0,27.900,0.0,16884.92400,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0
1,18.0,33.770,1.0,1725.55230,0.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0
2,28.0,33.000,3.0,4449.46200,0.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0
3,33.0,22.705,0.0,21984.47061,0.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0
4,32.0,28.880,0.0,3866.85520,0.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0


In [ ]:
get_dummy=pd.get_dummies(df_new)
get_dummy.head()

,age,bmi,children,charges,sex_female,sex_male,smoker_no,smoker_yes,region_northeast,region_northwest,region_southeast,region_southwest
0,19,27.900,0,16884.92400,1,0,0,1,0,0,0,1
1,18,33.770,1,1725.55230,0,1,1,0,0,0,1,0
2,28,33.000,3,4449.46200,0,1,1,0,0,0,1,0
3,33,22.705,0,21984.47061,0,1,1,0,0,1,0,0
4,32,28.880,0,3866.85520,0,1,1,0,0,1,0,0


In [ ]:
x=get_dummy.drop(["charges"],axis=1)
x.head()

,age,bmi,children,sex_female,sex_male,smoker_no,smoker_yes,region_northeast,region_northwest,region_southeast,region_southwest
0,19,27.900,0,1,0,0,1,0,0,0,1
1,18,33.770,1,0,1,1,0,0,0,1,0
2,28,33.000,3,0,1,1,0,0,0,1,0
3,33,22.705,0,0,1,1,0,0,1,0,0
4,32,28.880,0,0,1,1,0,0,1,0,0


In [ ]:
y=get_dummy[["charges"]]
y

,charges
0,16884.92400
1,1725.55230
2,4449.46200
3,21984.47061
4,3866.85520
...,...
1333,10600.54830
1334,2205.98080
1335,1629.83350
1336,2007.94500


In [ ]:
get_dummy.shape

(1199, 12)

#5.Split data into X (independent features) and Y(dependent features)

In [ ]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.2,random_state=5)

In [ ]:
from sklearn.linear_model import LinearRegression
model=LinearRegression().fit(x_train,y_train)

In [ ]:
pred=model.predict(x_test)
pred

In [ ]:
model.score(x_test,y_test)

0.6253302705767261

In [ ]:
1-(1-model.score(x_test,y_test))*(len(y_test)-1)/(len(y_test)-x_test.shape[1]-1)

0.60725409942034

#6.Build a simple Linear Regression model  andprint the residual plot and R2 score for test data(take test size as 0.2)


In [ ]:
fig,axes=plt.subplots(1,1,sharex=False,sharey=False)
fig.suptitle("[Residual plots]")
fig.set_size_inches(12,5)
axes.plot(model.predict(x_test),y_test-model.predict(x_test),'bo')
axes.axhline(y=0,color="k")
axes.grid()
axes.set_title("Linear")
axes.set_title("Predicted values")
axes.set_title("residuals")

Text(0.5, 1.0, 'residuals')

In [ ]:
from statsmodels.stats.outliers_influence import variance_inflation_factor
vif=pd.DataFrame()
vif["VIF"]=[variance_inflation_factor(x.values,i)for i in range(x.shape[1])]
vif["features"]=x.columns

/usr/local/lib/python3.10/dist-packages/statsmodels/stats/outliers_influence.py:198: RuntimeWarning: divide by zero encountered in double_scalars
  vif = 1. / (1. - r_squared_i)


In [ ]:
vif["VIF"]

0     1.020381
1     1.167918
2     1.003027
3          inf
4          inf
5          inf
6          inf
7          inf
8          inf
9          inf
10         inf
Name: VIF, dtype: float64

In [ ]:
from statsmodels.api import add_constant,OLS
x=add_constant(x)
temp_maxvif=5
max_vif=5
alt=x.copy()
count=1
highvif=[]
while temp_maxvif>=max_vif:
  temp_vif=pd.DataFrame()
  temp_vif["VIF"]=[variance_inflation_factor(alt.values,i)for i in range(alt.shape[1])]
  temp_vif["column_name"]=alt.columns
  temp_vif.dropna(inplace=True)
  tempcol=temp_vif.sort_values(["VIF"])[-1:]["column_name"].values[0]
  temp_maxvif=temp_vif.sort_values(["VIF"])[-1:]["VIF"].values[0]
  if(temp_maxvif >= max_vif):
    print(count)
    print(tempcol)
    alt=alt.drop(tempcol,axis=1)
    highvif.append(tempcol)

count=count+1




1
region_southwest
1
smoker_yes
1
sex_male
1
const
1
bmi
1
age


/usr/local/lib/python3.10/dist-packages/statsmodels/regression/linear_model.py:1781: RuntimeWarning: divide by zero encountered in double_scalars
  return 1 - self.ssr/self.centered_tss
/usr/local/lib/python3.10/dist-packages/statsmodels/stats/outliers_influence.py:198: RuntimeWarning: divide by zero encountered in double_scalars
  vif = 1. / (1. - r_squared_i)
/usr/local/lib/python3.10/dist-packages/statsmodels/regression/linear_model.py:1781: RuntimeWarning: divide by zero encountered in double_scalars
  return 1 - self.ssr/self.centered_tss
/usr/local/lib/python3.10/dist-packages/statsmodels/stats/outliers_influence.py:198: RuntimeWarning: divide by zero encountered in double_scalars
  vif = 1. / (1. - r_squared_i)
/usr/local/lib/python3.10/dist-packages/statsmodels/regression/linear_model.py:1781: RuntimeWarning: divide by zero encountered in double_scalars
  return 1 - self.ssr/self.centered_tss
/usr/local/lib/python3.10/dist-packages/statsmodels/stats/outliers_influence.py:198: R

In [ ]:
print(highvif)
highvif.remove('const')
print(highvif)
print(len(highvif))
x_new=x.drop(highvif,axis=1)
print(x.shape)

['region_southwest', 'smoker_yes', 'sex_male', 'const', 'bmi', 'age']
['region_southwest', 'smoker_yes', 'sex_male', 'bmi', 'age']
5
(1199, 12)


In [ ]:
x_new.head()

,const,children,sex_female,smoker_no,region_northeast,region_northwest,region_southeast
0,1.0,0,1,0,0,0,0
1,1.0,1,0,1,0,0,1
2,1.0,3,0,1,0,0,1
3,1.0,0,0,1,0,1,0
4,1.0,0,0,1,0,1,0


In [ ]:
x_train1,x_test1,y_train1,y_test1=train_test_split(x_new,y,test_size=0.20,random_state=10)
model_1=OLS(x_train1,y_train1)
modeldet=model_1.fit()
modeldet

In [ ]:
data=x_new.values.reshape(-1,1)
data

array([[1.],
       [0.],
       [1.],
       ...,
       [0.],
       [1.],
       [0.]])

In [ ]:
from sklearn.linear_model import Ridge
from sklearn.linear_model import Lasso

In [ ]:
x_train1,x_test1,y_train1,y_test1=train_test_split(x_new,y,test_size=0.20,random_state=10)
ridgereg=Ridge(alpha=0.0005)
ridgereg.fit(x_train1,y_train1)
pred=ridgereg.predict(x_test1)
score=ridgereg.score(x_test1,y_test1)
score

0.27960088173313813

In [ ]:
lassoreg=Lasso(alpha=0.03)
lassoreg.fit(x_train1,y_train)
pred=lassoreg.predict(x_test1)
lassoreg.score(x_test1,y_test1)

0.018375398287923117